### 벅스뮤직 실시간 차트 데이터 가져오기
- 웹 크롤링 기본적인 방법으로 데이터 연결(requests)

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import sys
import pandas as pd

In [12]:
url = 'https://music.bugs.co.kr/chart/track/realtime/total' #realtime

bugs_html = requests.get(url)

if bugs_html.status_code != 200:
    sys.exit('웹 페이지 정보를 가져올 수 없습니다.')
    
bugs_soup = bs(bugs_html.content, "html.parser")
bugs_soup =bugs_soup.find('tbody')
#print(bugs_soup)
tr_soup = bugs_soup.find_all('tr') #리스트형 구조로 'tr' 태크 전체 찾아옴


bugs_list=[]
# 순위, 곡명, 아티스트, 앨범
for tr in tr_soup:
    
    rank = tr.find('strong').text # 순위 값 추출
    title = tr.find('p', class_='title').text.replace("\n","")
    artist = tr.find('p', class_='artist').text.replace("\n","")
    album = tr.find('a', class_='album').text.replace("\n","")
    
    bugs_list.append({"순위" : rank, "곡명" : title, '아티스트' : artist, '앨범' : album})
    
df1=pd.DataFrame(bugs_list, columns = ['순위','앨범','곡명','아티스트'])
df1

,순위,앨범,곡명,아티스트
0,1,Butter / Permission to Dance,Permission to Dance,방탄소년단
1,2,Weekend,Weekend,태연 (TAEYEON)
2,3,MSG워너비 1집,바라만 본다,MSG워너비(M.O.M)
3,4,Bad Habits,Bad Habits,Ed Sheeran(에드 시런)
4,5,Next Level,Next Level,aespa
...,...,...,...,...
95,96,항해,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",AKMU (악동뮤지션)
96,97,The Prelude,At My Worst,Pink Sweat$
97,98,간 떨어지는 동거 OST Part.8,달,CHEEZE(치즈)
98,99,벌써일년,벌써일년,반하나\r반하나\r


###  멜론 차트 정보 가져오기

- selenium 모듈 사용
- selenuium 모듈은 chromedriver.exe를 이용해 크롬 브라우저 직접 컨트롤


In [2]:
#!pip install selenium

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import sys
import pandas as pd
import time

In [11]:
driver = webdriver.Chrome('c:/pydata/chromedriver.exe')
time.sleep(2) # time.sleep() :대기시간(초) 무조건 기다림(실행전)

url = 'https://www.melon.com/chart/index.htm' # chart1, /day, / week,/ month
driver.get(url) # 실행된 chromedriver 주소창에 url 입력 후 이동
driver.implicitly_wait(10) # 사이트 화면이 나타날때까지 기다림(최대 10초)
# 만약 10초가기다리면 error 발생

melon_html = driver.page_source # chromedriver 화면 전체 소스 코드 가져오기


    

In [12]:
#driver.get('https://www.naver.com') #네이버로 이동하고 싶은데

In [31]:
melon_soup = bs(melon_html ,  "html.parser")
melon_soup =melon_soup.find('tbody')
#print(bugs_soup)
tr_soup = melon_soup.find_all('tr') #리스트형 구조로 'tr' 태크 전체 찾아옴

melon_lst = []
cnt = 0       # 순위 표기
for tr in tr_soup:
    cnt += 1
    rank = cnt
    title = tr.find('div', class_='ellipsis rank01').get_text().replace('\n',"")
    artist = tr.find('span', class_='checkEllipsis').get_text().replace('\n',"")
    album = tr.find('div', class_='ellipsis rank03').get_text().replace('\n',"")
    starcnt = int(tr.find('span', class_='cnt').get_text().replace('\n총건수\n',"").replace(',',""))
    
    melon_lst.append([ rank, title, artist, album, starcnt])

df2 = pd.DataFrame(melon_lst, columns = ['순위', '곡명', '아티스트', '앨범', '좋아요'])
df2

,순위,곡명,아티스트,앨범,좋아요
0,1,바라만 본다,MSG워너비(M.O.M),MSG워너비 1집,107492
1,2,Next Level,aespa,Next Level,164456
2,3,Butter,방탄소년단,Butter,223920
3,4,Permission to Dance,방탄소년단,Butter / Permission to Dance,116380
4,5,신호등,이무진,신호등,110380
...,...,...,...,...,...
95,96,Love poem,아이유,Love poem,306280
96,97,Don't Start Now,Dua Lipa,Future Nostalgia,142121
97,98,내 마음이 움찔했던 순간 (취향저격 그녀 X 규현),규현 (KYUHYUN),내 마음이 움찔했던 순간 (취향저격 그녀 X 규현),106847
98,99,Hello Future,NCT DREAM,Hello Future - The 1st Album Repackage,67835


In [32]:
tr.find('div',class_='ellipsis rank01').get_text().replace('\n','')

'밤이 되니까'